### Обнаружение выбросов
Пусть будет эллипс, наблюдения внутри него будут 1, а вне -1

In [9]:
import numpy as np
import pandas as pd
from attr import dataclass
from sklearn.covariance import EllipticEnvelope
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

features, _ = make_blobs(n_samples=10, n_features=2, centers=1, random_state=1)
features[0,0] = 10000
features[0,1] = 10000

outlier_detector = EllipticEnvelope(contamination=0.1)
outlier_detector.fit(features)
outlier_detector.predict(features)

array([-1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

Если мы ожидаем, что наши данные будут иметь несколько выбросов, мы можем задать параметр contamination с каким-нибудь небольшим значением.<br>
Вместо того чтобы смотреть на наблюдения в целом, мы можем взглянуть на отдельные признаки и идентифицировать в этих признаках предельные значения, <br>
используя межквартильный размах (MKP, IQR)

In [10]:
feature = features[:,0]
def indicies_of_outliers(x):
    q1,q3 = np.percentile(x,[25,75])
    iqr = q3 - q1
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)
    return np.where((x >= upper_bound) & (x <= lower_bound))
indicies_of_outliers(feature)

(array([], dtype=int64),)

Единого наилучшего метода обнаружения выбросов не существует. Вместо этого у нас есть коллекция методов, все со своими преимуществами и недостатками, наилучшая стратегия использовать несколько методов и смотреть на результаты в целом. <br>
Возьмем набор данных о домах, в котором одним из признаков является количество комнат. Является ли выброс со 100 комнатами действительно домом или это отель который был неправильно классифицирован?

### Обработка выбросов
имеются выброс. Мы можем использовать 3 стратегии.<br>
Можем их отбросить.<br>
Пометить как выбросы и включить в качестве признака.<br>
Можем преобразовать признак, чтобы ослабить эффект выброса.

In [14]:
#Отбросить
houses = pd.DataFrame()
houses['Цена'] = [534433, 392333, 293222, 4322032]
houses['Ванные'] = [2, 3.5, 2, 116]
houses['Кв_футы'] = [1500, 2500, 1500, 48000]
houses[houses['Ванные']<20]

,Цена,Ванные,Кв_футы
0,534433,2.0,1500
1,392333,3.5,2500
2,293222,2.0,1500


In [16]:
#Пометить как выброс и включить
houses["Выброс"] = np.where(houses["Ванные"] < 20, 0, 1)
houses

,Цена,Ванные,Кв_футы,Выброс
0,534433,2.0,1500,0
1,392333,3.5,2500,0
2,293222,2.0,1500,0
3,4322032,116.0,48000,1


In [17]:
#Преобразовать признак, ослабить эффект выброса
houses["Логарифм_кв_футов"] = [np.log(x) for x in houses["Кв_футы"]]
houses

,Цена,Ванные,Кв_футы,Выброс,Логарифм_кв_футов
0,534433,2.0,1500,0,7.313220
1,392333,3.5,2500,0,7.824046
2,293222,2.0,1500,0,7.313220
3,4322032,116.0,48000,1,10.778956


Для начала нужно понять, что делать с данными выбросами. Это может быть:<br>
а) Ошибки в данных, из-за сломанного датчика или неверного значения, можем его исключить или заменить на NaN <br>
б) Являются подлинными предельными значениями, то более уместной будет маркировка их как выбросы или преобразование их значений <br><br>
Для полноты картины я бы использовал визуализацию выбросов - чтобы понять их структуру. Точка может быть нормальной по каждому признаку, но аномальной в совокупности (например, возраст 25, доход 1 млн/мес.)

### Группирование наблюдений с помощью кластеризации
Требуется сгруппировать наблюдения так, чтобы похожие наблюдения находились в одной группе

In [20]:
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

features, _ = make_blobs(n_samples=50, n_features=2, centers=3, random_state=1)
dataframe = pd.DataFrame(features, columns=["Признак_1", "Признак_2"])
clusterer = KMeans(3, random_state=0)
clusterer.fit(features)
dataframe["Группа"] = clusterer.predict(features)
dataframe.head(5)

,Признак_1,Признак_2,Группа
0,-9.877554,-3.336145,2
1,-7.287210,-8.353986,0
2,-6.943061,-7.023744,0
3,-7.440167,-8.791959,0
4,-6.641388,-8.075888,0


Кластеризация здесь используется как шаг предобработки. В частности для клатсеризации наблюдений в группы неконтролируемые (без учителя) обучающиеся алгоритмы, такие как метод k-средних

### Удаление наблюдений с пропущенными значениями

In [21]:
features = np.array([[1.1, 11.1],
                     [2.2, 22.2],
                     [3.3, 33.3],
                     [4.4, 44.4],
                     [np.nan, 55]])
features[~np.isnan(features).any(axis=1)]

array([[ 1.1, 11.1],
       [ 2.2, 22.2],
       [ 3.3, 33.3],
       [ 4.4, 44.4]])

Большинство машинно-обучающихся алгоритмов не могут обрабатывать пропущенные значения в массивах целей и признаков, по этой причине мы не можем игнорировать пропущенные значения в наших данных и должны решить эту проблему во время предобработки. <br><br>
Самым простым решением - удаление каждого наблюдения, содержащего одно или несколько пропущенных значений. <br><br>
Вместе с тем мы должны воздерживаться от удаления наблюдений с отсутствующими значениями. Их удаление является крайним средством, поскольку наш алгоритм теряет доступ к полезной информации, содержащейся в непропущенных значениях наблюдения. <br><br>Существует три типа пропущенных данных: <br>
1) MCAR - Missing Completely At Random
2) MAR - Missing At Random
3) MNAR - Missing Not At Random <br>

Иногда допустимо удалять MCAR или MAR, однако если значение MNAR, то факт, что значение пропущено, сам по себе является информацией. Удаление приводит данные к смещению, потому что мы удаляем наблюдения

### Импутация пропущенных значений
*импутация - процесс замещения пропущенных, некорректных или несостоятельных значений другими значениями*

In [25]:
#В учебнике упоминается - from fancyimpute import KNN - но он перестал работать после выпуска пособия
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
features, _ = make_blobs(n_samples=1000, n_features= 2, random_state=1)
scaler = StandardScaler()
standardized_features = scaler.fit_transform(features)
true_value = standardized_features[0, 0]
standardized_features[0, 0] = np.nan

# Импутировать пропущенные значения с помощью KNN
imputer = KNNImputer(n_neighbors=5)  # ← аналог KNN(k=5)
features_knn_imputed = imputer.fit_transform(standardized_features)

# Сравнить истинное и восстановленное значения
print("Истинное значение:", true_value)
print("Импутированное значение:", features_knn_imputed[0, 0])


Истинное значение: 0.8730186113995938
Импутированное значение: 1.0959262913919632


Недостаток алгоритма KNN: для того чтобы знать, какие наблюдения наиболее близки к пропущенному значению, необходимо вычислить расстояние между пропущенным
значением и каждым отдельным наблюдением. Это разумно в небольших наборах данных, но быстро становится проблематичным, если набор данных содержит миллионы наблюдений.